In [1]:
#importing necessary libraries
import requests
import pandas as pd
import numpy as np

In [3]:
#scrapping neighborhoods in Canada
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')

Page download successful


In [4]:
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head()

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
df_html.shape

(180, 3)

In [5]:
#Drop the the rows on which the Borough is empty
df_html.dropna(subset=['Borough'], inplace=True)

In [6]:
df_html.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
#Check Neighborhood is empty but Borough exists
n_empty_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]

In [8]:
n_empty_neighborhood

0

In [9]:
#Group by Postcode / Borough
df_postcodes = df_html.groupby(['Postal Code','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)])
df_postcodes.reset_index(inplace=True)
df_postcodes.head(5)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
df_postcodes.shape

(103, 3)

In [10]:
#Read CSV file from link and load into dataframe
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# Make sure both dataframes have the same 
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_postcodes.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [12]:
# Merge both datasets
df_Toronto_coordinates = pd.merge(df_postcodes, df_coordinates, on='PostalCode')
df_Toronto_coordinates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
#Export to .CSV
df_Toronto_coordinates.to_csv('Toronto_Coordinates.csv')

In [14]:
!conda install -c conda-forge folium=0.5.0

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [15]:
from sklearn.cluster import KMeans
import folium 

In [16]:
#count Bourough and Neighborhood
df_Toronto_coordinates.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [17]:
df_toronto = df_Toronto_coordinates[df_Toronto_coordinates['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [18]:
df_toronto.shape

(39, 5)

In [19]:
#Check the number of neighborhoods
print(df_toronto.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [20]:

#Create list with the Boroughs (to be used later)
boroughs = df_toronto['Borough'].unique().tolist()

In [21]:
#Obtain the coordinates from the dataset itself, just averaging Latitude/Longitude of the current dataset 
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [22]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

In [23]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [24]:
CLIENT_ID = '' # your Foursquare ID. Kept blank for Purpose, was executed with values
CLIENT_SECRET = '' # your Foursquare Secret, Kept blank for Purpose, was executed with values
VERSION = '20200715' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [27]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [28]:
#Number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,70,70,70,70,70,70
Christie,17,17,17,17,17,17
Church and Wellesley,71,71,71,71,71,71
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [37]:
#print out the list of categories
toronto_venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Greek Restaurant', 'Ice Cream Shop',
       'Italian Restaurant', 'Brewery', 'Juice Bar', 'Yoga Studio',
       'Restaurant', 'Fruit & Vegetable Store', 'Pizza Place',
       'Bookstore', 'Dessert Shop', 'Furniture / Home Store',
       'Bubble Tea Shop', 'Spa', 'Grocery Store', 'Café', 'Bakery',
       'Coffee Shop', 'Caribbean Restaurant', 'Indian Restaurant',
       'Lounge', 'American Restaurant', 'Frozen Yogurt Shop',
       'Sushi Restaurant', 'Liquor Store', 'Gym', 'Fast Food Restaurant',
       'Fish & Chips Shop', 'Park', 'Burrito Place', 'Pet Store',
       'Steakhouse', 'Movie Theater', 'Sandwich Place', 'Intersection',
       'Fish Market', 'Gay Bar', 'Cheese Shop', 'Seafood Restaurant',
       'Middle Eastern Restaurant', 'Stationery Store',
       'Comfort Food Restaurant', 'Thai Restaurant', 'Coworking Space',
       'Wine Bar', 'Latin American Restaurant', 'Gastropub',
       'Gym / Fi

In [29]:
# check if the results contain "Thai Restaurants"
#please note I changed the data to Thai because I was previously writing the code using Asian but the number is so small
"Indian Restaurant" in toronto_venues['Venue Category'].unique()

True

In [30]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1628, 230)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 230)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333
2,"Business reply mail Processing Centre, South C...",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.014286,0.000000,0.014286,0.000000,0.014286,0.000000,0.000000,0.014286
5,Christie,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.014085,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028169
7,"Commerce Court, Victoria Hotel",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,0.00,...,0.00000,0.00,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [32]:
len(to_grouped[to_grouped["Indian Restaurant"] > 0])

9

In [33]:
to_Indian = to_grouped[["Neighborhoods","Indian Restaurant"]]

In [35]:
to_Indian.shape

(39, 2)

## Cluster Neighborhoods

**Run k-means to cluster the neighborhoods in Toronto into 4 clusters.**

In [36]:
# set number of clusters
toclusters = 4

to_clustering = to_Indian.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 0, 1, 0, 1, 3, 1], dtype=int32)

In [37]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = to_Indian.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [38]:

to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head()


,Neighborhood,Indian Restaurant,Cluster Labels
0,Berczy Park,0.017544,0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,1
2,"Business reply mail Processing Centre, South C...",0.000000,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,1
4,Central Bay Street,0.014286,0


In [39]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1628, 9)


,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017544,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.017544,0,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.017544,0,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.017544,0,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.017544,0,43.644771,-79.373306,Goose Island Brewhouse,43.647329,-79.373541,Beer Bar


In [40]:
# sort the results by Cluster Labels
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(1628, 9)


,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017544,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Irish Embassy,43.647918,-79.377273,Irish Pub
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Carousel Bakery,43.648707,-79.371580,Bakery
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Starbucks,43.648738,-79.372519,Coffee Shop
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Olly Fresco's,43.646912,-79.379597,Deli / Bodega
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Cosmopolitan Toronto Centre Hotel & Spa,43.649064,-79.377598,Hotel
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Design Exchange,43.647972,-79.380104,Art Gallery
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,The Joint - Hogtown Smoke,43.649150,-79.374678,BBQ Joint
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,One King West Hotel & Residence,43.649139,-79.377876,Hotel


*Visualize the clusters*

In [41]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [42]:
# create map
map_clusters = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [43]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Study Clusters Individually

In [44]:
#Cluster 0
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017544,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Irish Embassy,43.647918,-79.377273,Irish Pub
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Carousel Bakery,43.648707,-79.371580,Bakery
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Starbucks,43.648738,-79.372519,Coffee Shop
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Olly Fresco's,43.646912,-79.379597,Deli / Bodega
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Cosmopolitan Toronto Centre Hotel & Spa,43.649064,-79.377598,Hotel
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,Design Exchange,43.647972,-79.380104,Art Gallery
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,The Joint - Hogtown Smoke,43.649150,-79.374678,BBQ Joint
31,Stn A PO Boxes,0.010204,0,43.646435,-79.374846,One King West Hotel & Residence,43.649139,-79.377876,Hotel


In [45]:
#Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
23,"Queen's Park, Ontario Provincial Government",0.0,1,43.662301,-79.389494,Queen's Park,43.663946,-79.392180,Park
25,"Richmond, Adelaide, King",0.0,1,43.650571,-79.384568,Cardio-Go,43.647017,-79.388143,Gym
25,"Richmond, Adelaide, King",0.0,1,43.650571,-79.384568,Earls Kitchen & Bar,43.647946,-79.383706,Bar
25,"Richmond, Adelaide, King",0.0,1,43.650571,-79.384568,Toronto Athletic Club,43.647360,-79.381177,Gym
25,"Richmond, Adelaide, King",0.0,1,43.650571,-79.384568,Smoke's Poutinerie,43.648325,-79.387646,Poutine Place
25,"Richmond, Adelaide, King",0.0,1,43.650571,-79.384568,Ruby Thai (First Canadian Place),43.649091,-79.381600,Thai Restaurant
25,"Richmond, Adelaide, King",0.0,1,43.650571,-79.384568,The Fifth & Terrace,43.649250,-79.389320,Modern European Restaurant
25,"Richmond, Adelaide, King",0.0,1,43.650571,-79.384568,Bymark,43.647217,-79.381252,Restaurant
25,"Richmond, Adelaide, King",0.0,1,43.650571,-79.384568,DoubleTree by Hilton,43.654608,-79.385942,Hotel
25,"Richmond, Adelaide, King",0.0,1,43.650571,-79.384568,Prairie Girl Bakery,43.648332,-79.382305,Cupcake Shop


In [46]:
#Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 2]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
34,"The Annex, North Midtown, Yorkville",0.047619,2,43.67271,-79.405678,Live Organic Food Bar,43.675053,-79.406715,Vegetarian / Vegan Restaurant
34,"The Annex, North Midtown, Yorkville",0.047619,2,43.67271,-79.405678,Madame Boeuf And Flea,43.675240,-79.406620,Burger Joint
34,"The Annex, North Midtown, Yorkville",0.047619,2,43.67271,-79.405678,Tim Hortons,43.675800,-79.403532,Coffee Shop
34,"The Annex, North Midtown, Yorkville",0.047619,2,43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
34,"The Annex, North Midtown, Yorkville",0.047619,2,43.67271,-79.405678,Martino's Pizza,43.675560,-79.403558,Pizza Place
34,"The Annex, North Midtown, Yorkville",0.047619,2,43.67271,-79.405678,Ezra's Pound,43.675153,-79.405858,Café
34,"The Annex, North Midtown, Yorkville",0.047619,2,43.67271,-79.405678,Jean Sibelius Square,43.671426,-79.408831,Park
34,"The Annex, North Midtown, Yorkville",0.047619,2,43.67271,-79.405678,Big Crow,43.675896,-79.403680,BBQ Joint
34,"The Annex, North Midtown, Yorkville",0.047619,2,43.67271,-79.405678,Krispy Kreme Doughnut Cafe,43.674732,-79.407730,Donut Shop
34,"The Annex, North Midtown, Yorkville",0.047619,2,43.67271,-79.405678,Creeds Coffee Bar,43.674100,-79.410838,Coffee Shop


In [47]:
#Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 3]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
30,"St. James Town, Cabbagetown",0.022727,3,43.667967,-79.367675,Kanpai Snack Bar,43.664331,-79.368065,Taiwanese Restaurant
30,"St. James Town, Cabbagetown",0.022727,3,43.667967,-79.367675,Wellesley Park,43.669649,-79.362155,Park
30,"St. James Town, Cabbagetown",0.022727,3,43.667967,-79.367675,The Epicure Shop,43.663965,-79.367795,Deli / Bodega
30,"St. James Town, Cabbagetown",0.022727,3,43.667967,-79.367675,Labour Of Love,43.663907,-79.368822,Gift Shop
30,"St. James Town, Cabbagetown",0.022727,3,43.667967,-79.367675,Mr. Jerk,43.667328,-79.373389,Caribbean Restaurant
30,"St. James Town, Cabbagetown",0.022727,3,43.667967,-79.367675,Pet Valu,43.664205,-79.368460,Pet Store
30,"St. James Town, Cabbagetown",0.022727,3,43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
30,"St. James Town, Cabbagetown",0.022727,3,43.667967,-79.367675,St. Jamestown Delicatessen,43.665811,-79.368648,Butcher
30,"St. James Town, Cabbagetown",0.022727,3,43.667967,-79.367675,Toronto Dance Theatre,43.666232,-79.367075,General Entertainment
30,"St. James Town, Cabbagetown",0.022727,3,43.667967,-79.367675,Fair Trade Jewellery Co.,43.665348,-79.368362,Jewelry Store


## Conclusion

There are no Indian Restaurant in Cluster 1, which includes neighborhood like `Richmond, Adelaide, King, Parkdale, Queen's park, Forest Hill, etc`. 
Cluster 2 and 3 have good count of Indian Restaurants, whereas Cluster 0 has some restaurants in that neighborhood. 

Studying the above cluster, I would recommend the food chain restaurant to open an Indian restaurant in Cluster 1 with little to no competition. 